In [4]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, assemble, Aer
import torch
from torch import nn
from math import pi
import pandas as pd
import numpy as np

In [29]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.distributions.Uniform(0, 0.01).sample((4,3)))
        self.bias = nn.Parameter(torch.zeros(4))
        self.circuit_size = 4
        self.n_actions = 4
        self.shots = 1000
    
    def _circuitMaker(self, n_qubits, encodings, params, measurement, n_cbits=1):
        # create quantum circuit
        qr = QuantumRegister(n_qubits, 'qubit')
        cr = ClassicalRegister(n_cbits, 'cr')
        qc = QuantumCircuit()
        qc.add_register(qr)
        qc.add_register(cr)
        
        # encoding block
        encoder0, encoder1 = self._encoder(encodings)
        # print(encodings, encoder1)
        # qc.rx(pi, encoder0)
        # qc.rz(pi, encoder0)
        if encoder1:
            qc.rx(pi, encoder1)
            qc.rz(pi, encoder1)
            qc.barrier(qr)
        
        for i in range(n_qubits):
            # entangle block
            if i < n_qubits - 1:
                qc.cnot(i, i+1)
            else:
                qc.barrier(qr)

        for i in range(n_qubits):
            # variational block
            qc.u3(params[i][0], params[i][1], params[i][2], i)

        # measurement
        for i, j in enumerate(measurement):
            qc.measure(qr[j], cr[i])
        
        return qc

    def _encoder(self, encodings):
        return [i for i, b in enumerate(encodings) if b == '0'], [i for i, b in enumerate(encodings) if b == '1']

    def _getAction_single_measure(self, state):
        b_state = f'{state:0{self.circuit_size}b}'
        # tasks = [circuitMaker(circuit_size, b_state, parameters, [i]) for i in range(n_actions)]
        aer_sim = Aer.get_backend('aer_simulator')
        param_current = self.weights.detach().numpy()
        rlt = [aer_sim.run(assemble(self._circuitMaker(self.circuit_size, b_state, param_current, [i]), shots=self.shots)).result().get_counts() for i in range(self.n_actions)]
        # qobj = assemble(qc, shots=shots)
        # job = aer_sim.run(qobj)
        # rlt = job.result().get_counts()
        # print(rlt)
        q_exps = [(i['0']-i['1'])/self.shots if len(i) > 1 else (-i['1']/self.shots if '1' in i.keys() else i['0']/self.shots) for i in rlt]

        return [a + b for a, b in zip(q_exps, self.bias.detach().numpy())]

    def forward(self, stac):
        return torch.tensor([self._getAction_single_measure(state)[action] for state, action in stac])

sampled_data = {
    'state':[0, 3, 4, 8, 14], 
    'action':[1, 0, 2, 3, 2], 
    'reward':[-0.01, -0.01, -0.2, -0.01, 1], 
    'new_state':[4, 2, 5, 4, 15], 
    'done':[0, 0, 1, 0, 1]
    }

sampled_df = pd.DataFrame(sampled_data)
model_pred = Model()
model_tar = Model()
opt = torch.optim.Adam(params=model_pred.parameters(), lr=0.1)



In [30]:
GAMMA = 0.1
# TODO: check how to define class model_tar->forward->return pytorch.tensor [(sampled_df['reward'][i] if sampled_df['done'][i] else sampled_df['reward'][i] + GAMMA*np.max(tar)) for i in sampled_df.index]
# check this way:https://towardsdatascience.com/integrating-tensorflow-and-qiskit-for-quantum-machine-learning-7fa6b14d5294
# this:https://medium.com/qiskit/introducing-qiskit-machine-learning-5f06b6597526
# (https://www.reddit.com/r/QuantumComputing/comments/mm5urd/qiskit_with_pytorch_has_arrived/)
# First see:
# https://github.com/Qiskit/qiskit-machine-learning
# https://qiskit.org/documentation/machine-learning/tutorials/05_torch_connector.html
pred = model_pred.forward(zip(sampled_df['state'], sampled_df['action']))
# tar = model_tar.forward(zip(sampled_df['state'], sampled_df['action']))
# tar = [(sampled_df['reward'][i] if sampled_df['done'][i] else sampled_df['reward'][i] + GAMMA*np.max(tar)) for i in sampled_df.index]

for name, param in model_pred.named_parameters():
    if param.requires_grad:
        print(name, param.data)

loss_fn = nn.MSELoss()
loss = loss_fn(pred, pred)
loss.backward()
opt.step()
opt.zero_grad()

print('after one step optimize')

for name, param in model_pred.named_parameters():
    if param.requires_grad:
        print(name, param.data)

weights tensor([[0.0077, 0.0034, 0.0029],
        [0.0023, 0.0005, 0.0048],
        [0.0073, 0.0039, 0.0048],
        [0.0044, 0.0051, 0.0003]])
bias tensor([0., 0., 0., 0.])


C:\Users\XINYAN~1\AppData\Local\Temp/ipykernel_18524/1584971010.py:37: DeprecationWarning: The QuantumCircuit.u3 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use QuantumCircuit.u instead, which acts identically. Alternatively, you can decompose u3 in terms of QuantumCircuit.p and QuantumCircuit.sx: u3(ϴ,φ,λ) = p(φ+π) sx p(ϴ+π) sx p(λ) (2 pulses on hardware).
  qc.u3(params[i][0], params[i][1], params[i][2], i)


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [18]:
weights = torch.distributions.Uniform(0, 0.01).sample((4,3))
bias = torch.zeros(4)
##params_q = nn.Parameter(weights)
params_bias = nn.Parameter(bias)
print(params_bias.detach().numpy())
#print(params_q.detach().numpy()) # CPU tensor
# print(params_q.detach().to('cpu').numpy()) # CUDA tensor

[0. 0. 0. 0.]


In [ ]:


def cost(params, sampled_vs, q_target):
    # global sampled_vs, q_target
    v_targets = [(s[2] if s[4] else s[2] + GAMMA*np.max(q_target)) for s in sampled_vs]
    v_predics = [getAction_single_measure(s[0], params)[s[1]] for s in sampled_vs]
    cost = sum([(tar - pred)**2 for tar, pred in zip(v_targets, v_predics)])/MINIBATCH # MSE
    # print('cost: ', cost)
    return cost

n_qubits = 4
n_cbits = 1
n_actions = 4
circuit_size = 4
shots = 1000
GAMMA = 0.9
MINIBATCH = 5

qc = circuitMaker(n_qubits, '1011', [1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0], [0])
qc.draw()